<a href="https://colab.research.google.com/github/HaroonMalik771/Heart-Disease-Prediction-from-ECG-through-Image--Processing/blob/main/project_notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### IMPORTING LIBRARIES

In [17]:
import pandas as pd
import numpy as np
import os
from natsort import natsorted
import joblib
import sklearn

In [1]:
!pip install scikit-learn==1.3.2


### **WORKING ON COMBING MULTIPLE LEAD FILES**

#### **COMBINED CSV OF EACH LEAD(1-12) FROM ALL IMAGES**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#now reading just lead1
df=pd.read_csv('/content/drive/MyDrive/Project/Combined1d_csv/Combined1d_csv/Combined_IDLead_1.csv')
df['Target'].unique()

array(['No', 'HB', 'MI', 'PM'], dtype=object)

In [19]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [20]:
#convert Target column values as Numeric using ngroups
encode_target_label = df.groupby('Target').ngroup().rename("target").to_frame()
test_final  = df.merge(encode_target_label, left_index=True, right_index=True)
test_final.drop(columns=['Target'],inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,target
0,0.728449,0.680755,0.619010,0.645367,0.681570,0.732488,0.758448,0.750660,0.728282,0.707928,...,0.637260,0.664539,0.667226,0.637064,0.593287,0.545503,0.515049,0.563257,0.633581,2
1,0.957972,0.950695,0.941024,0.930501,0.913601,0.892244,0.868016,0.855127,0.835307,0.798640,...,0.778790,0.806883,0.818640,0.842472,0.866740,0.884152,0.897196,0.911293,0.922903,2
2,0.611084,0.661575,0.695790,0.741113,0.716666,0.595794,0.425022,0.286457,0.425022,0.611384,...,0.000000,0.042690,0.165850,0.363445,0.549460,0.539346,0.522272,0.491668,0.454949,2
3,0.839213,0.861690,0.866457,0.865756,0.855027,0.855606,0.845561,0.843187,0.846784,0.824438,...,0.789156,0.793622,0.787665,0.794515,0.796739,0.804063,0.809944,0.801814,0.777322,2
4,0.917753,0.924369,0.873765,0.791381,0.699513,0.604927,0.500312,0.446012,0.528910,0.634068,...,0.200676,0.300147,0.407225,0.507346,0.605953,0.699309,0.790334,0.856593,0.849957,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,0.874246,0.877014,0.864280,0.860505,0.871349,0.912404,0.958148,0.977826,0.956314,0.926773,...,0.908312,0.926328,0.898749,0.855709,0.823132,0.815458,0.818083,0.829300,0.822382,3
924,0.829815,0.832084,0.852396,0.909665,0.988242,1.000000,0.923323,0.821865,0.721302,0.612039,...,0.429721,0.531567,0.642137,0.742063,0.833042,0.814867,0.777622,0.760714,0.759294,3
925,0.469048,0.417983,0.362322,0.351995,0.391493,0.418305,0.440135,0.444598,0.460402,0.506810,...,0.408587,0.401864,0.387069,0.359590,0.325879,0.288894,0.293521,0.344504,0.399012,3
926,0.682510,0.682286,0.641051,0.620212,0.608210,0.576331,0.603596,0.645714,0.677964,0.720297,...,0.452247,0.450421,0.439278,0.439086,0.394417,0.441650,0.473909,0.539199,0.547146,3


#### **PERFORM DIMENSIONALITY REDUCTION JUST FOR CHECKING/UNDERSTANDING**

In [21]:
#just for testing
# Now Perform Dimensionality reduction (PCA) on that Dataframe and check
from sklearn.decomposition import PCA

#do PCA and choose componeents as 100
pca = PCA(n_components=100)
x_pca = pca.fit_transform(test_final.iloc[:,0:-1])
x_pca = pd.DataFrame(x_pca)

# Calculate the variance explained by priciple components
explained_variance = pca.explained_variance_ratio_
print('Variance of each component:', pca.explained_variance_ratio_)
print('\n Total Variance Explained:', round(sum(list(pca.explained_variance_ratio_))*100, 2))

#store the new pca generated dimensions in a dataframe
pca_df = pd.DataFrame(data = x_pca)
target = pd.Series(test_final['target'], name='target')
result_df = pd.concat([pca_df, target], axis=1)
result_df

Variance of each component: [1.76145888e-01 9.50265614e-02 6.99060614e-02 6.15960001e-02
 5.34876630e-02 4.23664893e-02 3.68320213e-02 3.38541791e-02
 3.00884979e-02 2.90396728e-02 2.64962509e-02 2.42272738e-02
 2.10221030e-02 1.99751559e-02 1.77321042e-02 1.63016802e-02
 1.53898622e-02 1.48412074e-02 1.33644825e-02 1.19674074e-02
 1.16813409e-02 1.05807650e-02 9.68875480e-03 9.47385060e-03
 8.65347748e-03 8.47506998e-03 7.93382172e-03 7.30163338e-03
 6.76380665e-03 6.36886390e-03 6.02004791e-03 5.46823032e-03
 5.31229911e-03 4.97821789e-03 4.74686092e-03 4.46081684e-03
 4.21254684e-03 4.01200243e-03 3.87246476e-03 3.52519084e-03
 3.37596894e-03 3.26978336e-03 3.08241145e-03 2.96423495e-03
 2.73419816e-03 2.50965698e-03 2.35335480e-03 2.25665349e-03
 2.20141761e-03 1.96782025e-03 1.74343954e-03 1.70982830e-03
 1.57456047e-03 1.53704487e-03 1.36768435e-03 1.33167096e-03
 1.26444173e-03 1.20053330e-03 1.18738749e-03 1.08864087e-03
 1.02824532e-03 9.11484783e-04 7.89962329e-04 7.59785111e

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,1.018578,1.148263,-0.589582,0.193617,0.047950,-0.309400,-0.161566,0.478471,0.972403,-0.031832,...,0.011767,-0.004017,0.024376,0.012375,-0.001681,-0.000389,0.002912,-0.063222,-0.029497,2
1,-1.098692,0.289832,-1.766388,1.076165,-0.261201,-0.820446,-0.474188,-0.515238,0.692389,1.501606,...,-0.017417,0.060519,-0.007405,0.013418,0.000529,0.044967,0.031136,-0.037157,0.001911,2
2,0.275021,-0.451289,0.106750,-0.426415,0.066133,0.692474,0.634894,-0.035867,0.815855,-0.909473,...,0.045448,-0.042226,-0.030141,0.117641,-0.046180,-0.020235,-0.020073,0.046146,0.005645,2
3,-1.517085,1.662693,-1.021167,0.804267,-0.281985,0.518180,0.355748,-0.344235,-0.910867,-0.629517,...,0.000717,0.041107,0.018190,-0.049882,-0.038503,-0.016101,-0.020183,0.026926,-0.036576,2
4,-0.152840,-1.046283,0.351278,1.100381,-1.613642,1.484188,-0.113277,-0.251152,0.179023,-0.233104,...,-0.010029,0.054449,0.000955,0.031419,0.013509,-0.003714,0.041833,0.000824,-0.026847,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,-1.321884,2.153021,0.788596,-1.304253,0.458186,-0.859346,-0.069127,-0.392796,0.755732,-0.584050,...,-0.036580,0.076983,-0.044319,-0.034817,0.008062,-0.022643,-0.049445,-0.011718,0.013077,3
924,-0.867163,-0.040504,0.940680,0.302648,-0.469672,-0.368255,1.065579,0.801522,0.690113,0.953008,...,0.012292,-0.092672,0.013184,-0.028048,-0.009013,0.057285,-0.010481,-0.066483,-0.017723,3
925,3.753012,0.841636,-0.317393,-0.296117,0.593769,-0.255474,-0.057091,-0.072048,0.664386,-0.837668,...,-0.008894,-0.005923,-0.002683,0.008653,-0.021819,0.063855,0.020538,-0.002534,-0.017404,3
926,0.603083,0.126259,0.003433,0.283612,0.169559,-0.156326,-0.068399,-0.184308,0.461063,-0.002047,...,0.057295,0.024621,-0.011203,-0.077137,-0.001288,0.031111,-0.053394,0.023715,0.062386,3


In [22]:
result_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,1.018578,1.148263,-0.589582,0.193617,0.047950,-0.309400,-0.161566,0.478471,0.972403,-0.031832,...,0.011767,-0.004017,0.024376,0.012375,-0.001681,-0.000389,0.002912,-0.063222,-0.029497,2
1,-1.098692,0.289832,-1.766388,1.076165,-0.261201,-0.820446,-0.474188,-0.515238,0.692389,1.501606,...,-0.017417,0.060519,-0.007405,0.013418,0.000529,0.044967,0.031136,-0.037157,0.001911,2
2,0.275021,-0.451289,0.106750,-0.426415,0.066133,0.692474,0.634894,-0.035867,0.815855,-0.909473,...,0.045448,-0.042226,-0.030141,0.117641,-0.046180,-0.020235,-0.020073,0.046146,0.005645,2
3,-1.517085,1.662693,-1.021167,0.804267,-0.281985,0.518180,0.355748,-0.344235,-0.910867,-0.629517,...,0.000717,0.041107,0.018190,-0.049882,-0.038503,-0.016101,-0.020183,0.026926,-0.036576,2
4,-0.152840,-1.046283,0.351278,1.100381,-1.613642,1.484188,-0.113277,-0.251152,0.179023,-0.233104,...,-0.010029,0.054449,0.000955,0.031419,0.013509,-0.003714,0.041833,0.000824,-0.026847,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,-1.321884,2.153021,0.788596,-1.304253,0.458186,-0.859346,-0.069127,-0.392796,0.755732,-0.584050,...,-0.036580,0.076983,-0.044319,-0.034817,0.008062,-0.022643,-0.049445,-0.011718,0.013077,3
924,-0.867163,-0.040504,0.940680,0.302648,-0.469672,-0.368255,1.065579,0.801522,0.690113,0.953008,...,0.012292,-0.092672,0.013184,-0.028048,-0.009013,0.057285,-0.010481,-0.066483,-0.017723,3
925,3.753012,0.841636,-0.317393,-0.296117,0.593769,-0.255474,-0.057091,-0.072048,0.664386,-0.837668,...,-0.008894,-0.005923,-0.002683,0.008653,-0.021819,0.063855,0.020538,-0.002534,-0.017404,3
926,0.603083,0.126259,0.003433,0.283612,0.169559,-0.156326,-0.068399,-0.184308,0.461063,-0.002047,...,0.057295,0.024621,-0.011203,-0.077137,-0.001288,0.031111,-0.053394,0.023715,0.062386,3


#### **TRYING DIFFERENT ML MODELS ON A SINGLE LEAD(EX : 1) POST DIMENSIONALITY REDUCTION**

##### **KNN**

In [23]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('knn', KNeighborsClassifier())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# have paased less range value of hyperparamter since i'm using free tier version of google colab.
k_range = list(range(1, 9))
parameters = dict(knn__n_neighbors=k_range)

#input
X = result_df.iloc[:,0:-1]

#target
y=result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#increasing cv score takes lot of time in gooogle colab, so kept it just 2.
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

Knn_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
print("Accuracy: {}".format(Knn_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.782258064516129
              precision    recall  f1-score   support

           0       0.87      0.63      0.73       105
           1       0.91      0.91      0.91        94
           2       0.72      0.88      0.79       112
           3       0.63      0.67      0.65        61

    accuracy                           0.78       372
   macro avg       0.78      0.77      0.77       372
weighted avg       0.80      0.78      0.78       372

Tuned Model Parameters: {'knn__n_neighbors': 1}


##### **LOGISTIC REGRESSION**

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('lr', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

#input
X = result_df.iloc[:,0:-1]

#target
y=result_df.iloc[:,-1]

#parameters for gridsearchcv if we increase range of entries from 5 to higher value, we can get greater accurange
c_space = np.logspace(-4, 4, 3)
parameters = {'lr__C': c_space,'lr__penalty': ['l2']}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#call GridSearchCV and set crossvalscore to 2
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)
LR_Accuracy = cv.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [25]:
# Compute and print metrics
print("Accuracy: {}".format(LR_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.5510752688172043
              precision    recall  f1-score   support

           0       0.37      0.32      0.35       105
           1       0.73      0.91      0.81        94
           2       0.56      0.60      0.58       112
           3       0.41      0.30      0.34        61

    accuracy                           0.55       372
   macro avg       0.52      0.53      0.52       372
weighted avg       0.53      0.55      0.53       372

Tuned Model Parameters: {'lr__C': 10000.0, 'lr__penalty': 'l2'}


##### **SVM**

In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline
steps = [('SVM', SVC())]

pipeline = Pipeline(steps)

#input
X = result_df.iloc[:,0:-1]

#target
y=result_df.iloc[:,-1]

# Specify the hyperparameter space, if we increase the penalty(c) and gamma value the accurancy can be increased.
#since it takes lots of time in google colab provided only a single value
parameters =  {'SVM__C':[10],'SVM__gamma':[1]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=21)

cv = GridSearchCV(pipeline,parameters,cv=3)
cv.fit(X_train,y_train)

y_pred = cv.predict(X_test)
SVM_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
SVM_Accuracy=cv.score(X_test, y_test)

print("Accuracy: {}".format(SVM_Accuracy))
print(classification_report(y_test, y_pred))

Accuracy: 0.8225806451612904
              precision    recall  f1-score   support

           0       0.58      1.00      0.74        93
           1       1.00      1.00      1.00        99
           2       1.00      0.61      0.76       117
           3       1.00      0.68      0.81        63

    accuracy                           0.82       372
   macro avg       0.90      0.82      0.83       372
weighted avg       0.90      0.82      0.83       372



#### **NOW COMBINING ALL 12 LEADS INTO A SINGLE CSV FILE AND THEN PERFROM MODEL ANALYSIS**


In [29]:
#lets try combining all 12 leads in a single csv
location= '/content/drive/MyDrive/Project/Combined1d_csv/Combined1d_csv'
for files in natsorted(os.listdir(location)):
  if files.endswith(".csv") and not files.endswith("13.csv"):
    if files!='Combined_IDLead_1.csv':
      df=pd.read_csv('/content/drive/MyDrive/Project/Combined1d_csv/Combined1d_csv/{}'.format(files))
      df.drop(columns=['Unnamed: 0'],inplace=True)
      test_final=pd.concat([test_final,df],axis=1,ignore_index=True)
      test_final.drop(columns=test_final.columns[-1],axis=1,inplace=True)

#drop the target column
test_final.drop(columns=[255],axis=1,inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,...,5855,5856,5857,5858,5859,5860,5861,5862,5863,5864
0,0.728449,0.680755,0.619010,0.645367,0.681570,0.732488,0.758448,0.750660,0.728282,0.707928,...,0.864067,0.849256,0.854949,0.861380,0.875514,0.868763,0.847450,0.805689,0.751761,0.702102
1,0.957972,0.950695,0.941024,0.930501,0.913601,0.892244,0.868016,0.855127,0.835307,0.798640,...,0.925865,0.928285,0.946033,0.947274,0.946394,0.936536,0.920869,0.910320,0.905436,0.876942
2,0.611084,0.661575,0.695790,0.741113,0.716666,0.595794,0.425022,0.286457,0.425022,0.611384,...,0.170137,0.166206,0.207633,0.258184,0.286993,0.304742,0.325659,0.361189,0.451946,0.543373
3,0.839213,0.861690,0.866457,0.865756,0.855027,0.855606,0.845561,0.843187,0.846784,0.824438,...,0.880043,0.883833,0.870995,0.861323,0.864892,0.863552,0.839506,0.805486,0.801828,0.826618
4,0.917753,0.924369,0.873765,0.791381,0.699513,0.604927,0.500312,0.446012,0.528910,0.634068,...,0.400774,0.380920,0.439510,0.505257,0.561538,0.577997,0.566082,0.547642,0.538735,0.527560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,0.874246,0.877014,0.864280,0.860505,0.871349,0.912404,0.958148,0.977826,0.956314,0.926773,...,0.785103,0.676795,0.579054,0.476613,0.458748,0.565470,0.681896,0.792646,0.871660,0.872789
924,0.829815,0.832084,0.852396,0.909665,0.988242,1.000000,0.923323,0.821865,0.721302,0.612039,...,0.913404,0.968015,0.992614,0.945789,0.876660,0.808906,0.741645,0.736615,0.797729,0.855637
925,0.469048,0.417983,0.362322,0.351995,0.391493,0.418305,0.440135,0.444598,0.460402,0.506810,...,0.730354,0.697465,0.714527,0.745605,0.754952,0.755059,0.755059,0.755093,0.759093,0.767555
926,0.682510,0.682286,0.641051,0.620212,0.608210,0.576331,0.603596,0.645714,0.677964,0.720297,...,0.860939,0.824976,0.783459,0.761391,0.741917,0.770631,0.802701,0.821503,0.846300,0.858795


In [30]:
#write the final file to csv
test_final.to_csv('final_1D.csv',header=False,index=False)

#### **TEST DIMENSIONALITY REDUCTION EXPLAINED VARIANCE ON  THE DATA**


In [31]:
# Now Perform Dimensionality reduction (PCA) on that Dataframe and check
from sklearn.decomposition import PCA

#do PCA and choose componeents as 400
pca = PCA(n_components=400)
x_pca = pca.fit_transform(test_final)
x_pca = pd.DataFrame(x_pca)

# Calculate the variance explained by priciple components
explained_variance = pca.explained_variance_ratio_
print('Variance of each component:', pca.explained_variance_ratio_)
print('\n Total Variance Explained:', round(sum(list(pca.explained_variance_ratio_))*100, 2))

#store the new pca generated dimensions in a dataframe
#store the new pca generated dimensions in a dataframe
pca_df = pd.DataFrame(data = x_pca)
target = pd.Series(result_df.iloc[:,-1], name='target')
final_result_df = pd.concat([pca_df, target], axis=1)
final_result_df




Variance of each component: [8.20258548e-02 4.66380480e-02 3.86082026e-02 3.02049312e-02
 2.59833872e-02 2.38307800e-02 2.17883512e-02 2.07675606e-02
 1.97729319e-02 1.79354562e-02 1.67284357e-02 1.55312283e-02
 1.52391916e-02 1.44456920e-02 1.36100667e-02 1.35356330e-02
 1.27686285e-02 1.25275640e-02 1.18640264e-02 1.12874872e-02
 1.08991496e-02 1.06149126e-02 1.04909225e-02 1.02297055e-02
 9.80159368e-03 9.42157394e-03 9.07406578e-03 8.71885466e-03
 8.38480474e-03 8.19108157e-03 7.97941775e-03 7.76684061e-03
 7.50092308e-03 7.19123408e-03 7.06158992e-03 6.91913717e-03
 6.77144458e-03 6.55907255e-03 6.40309636e-03 6.29435108e-03
 6.02825668e-03 5.84227310e-03 5.75548371e-03 5.71197863e-03
 5.60015540e-03 5.43404219e-03 5.32462897e-03 5.12045374e-03
 5.08653932e-03 4.93291575e-03 4.78578036e-03 4.58517927e-03
 4.47874045e-03 4.45012310e-03 4.30515950e-03 4.21542751e-03
 4.10714839e-03 4.08627368e-03 4.01292909e-03 3.92982254e-03
 3.88032615e-03 3.78941638e-03 3.74803922e-03 3.66710355e

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,target
0,-0.777652,0.346732,-1.321652,-0.039334,0.430210,2.602178,0.589793,-1.732037,-3.267205,-1.635752,...,-0.147507,-0.079652,-0.038653,-0.267092,0.023138,0.205084,0.117108,-0.108007,-0.050739,2
1,-1.817704,-5.430882,1.537896,1.796222,-0.350355,1.128484,-0.456832,1.713835,-6.807842,-0.975054,...,-0.159727,-0.086878,-0.075419,0.087139,-0.007383,0.163407,0.007185,0.143001,0.017076,2
2,9.242464,0.263155,-3.225174,-1.110968,-4.261920,-3.017125,2.796778,4.159151,1.638678,1.654743,...,0.068920,0.024957,-0.084439,-0.056365,0.033393,-0.121878,0.067796,-0.129895,-0.057364,2
3,-6.364148,-1.295384,-1.871954,-2.249239,1.502461,2.863616,4.801614,-2.173422,-3.151378,-5.358400,...,-0.060691,0.188644,-0.090644,0.155832,0.131171,-0.075046,0.038052,-0.190430,0.004960,2
4,-5.335609,-4.335754,-0.465859,-0.762277,0.129209,2.959243,2.937732,-0.241102,-2.354260,4.315757,...,0.129871,0.037771,-0.205198,0.107970,0.284350,0.007448,0.258970,-0.206565,0.210498,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,-4.626070,-3.674085,-0.883185,0.875895,2.602038,3.034868,4.919216,3.258500,0.280402,1.810444,...,-0.077720,-0.063606,-0.111896,-0.048276,-0.192246,-0.044257,0.158065,0.172566,-0.065066,3
924,-8.948774,0.483759,-4.604114,4.035889,3.899770,-2.144356,2.156369,0.987695,-0.914935,0.554256,...,-0.387596,-0.054421,0.097340,-0.008348,-0.149861,-0.193229,0.010366,0.132648,0.071542,3
925,-3.537766,5.736557,6.730076,3.625789,2.111307,1.513954,-1.828985,2.726393,-0.568539,-3.222119,...,-0.012983,0.218342,0.215757,0.134993,0.102029,0.068352,0.014546,0.140234,0.093409,3
926,-1.847042,10.152251,3.413474,-0.734832,-0.302906,1.368098,-2.762880,0.340497,-0.526940,-1.516321,...,0.067903,-0.186224,-0.261159,-0.082185,-0.066036,-0.211206,0.102089,0.064101,-0.109633,3


In [32]:
#save to dimensionally reduced csv file
final_result_df.to_csv("pca_final.csv")

In [33]:
import joblib
#save the PCA model
joblib_file='PCA_ECG.pkl'
joblib.dump(pca,joblib_file)

['PCA_ECG.pkl']

#### **TRYING DIFFERENT ML MODELS ON THE ALL 12 LEADS COMBINED FILE WITHOUT DIMENSIONALITY REDUCTION**

##### **KNN**


In [39]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('knn', KNeighborsClassifier())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# have paased less range value of hyperparamter since i'm using free tier version of google colab.
k_range = list(range(1, 30))
parameters = dict(knn__n_neighbors=k_range)

#input
X = final_result_df.iloc[:,:-1]

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#increasing cv score takes lot of time in gooogle colab, so kept it just 2.
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

Knn_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
print("Accuracy: {}".format(Knn_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.7956989247311828
              precision    recall  f1-score   support

           0       0.92      0.66      0.77       105
           1       0.95      0.91      0.93        94
           2       0.70      0.86      0.77       112
           3       0.65      0.74      0.69        61

    accuracy                           0.80       372
   macro avg       0.80      0.79      0.79       372
weighted avg       0.82      0.80      0.80       372

Tuned Model Parameters: {'knn__n_neighbors': 1}


##### **LOGISTIC REGRESSION**


In [40]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('lr', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

#input
X = final_result_df.iloc[:,:-1]

#target
y=final_result_df.iloc[:,-1]

#parameters for gridsearchcv
c_space = np.logspace(-4, 4, 10)
parameters = {'lr__C': c_space,'lr__penalty': ['l2']}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#call GridSearchCV and set crossvalscore to 2
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)
LR_Accuracy = cv.score(X_test, y_test)

In [41]:
# Compute and print metrics
print("Accuracy: {}".format(LR_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.7903225806451613
              precision    recall  f1-score   support

           0       0.88      0.60      0.71       105
           1       0.83      0.91      0.87        94
           2       0.82      0.87      0.84       112
           3       0.62      0.79      0.69        61

    accuracy                           0.79       372
   macro avg       0.79      0.79      0.78       372
weighted avg       0.80      0.79      0.79       372

Tuned Model Parameters: {'lr__C': 0.046415888336127774, 'lr__penalty': 'l2'}


##### **SVM**


In [42]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline
steps = [('SVM', SVC())]

pipeline = Pipeline(steps)

#input
X = final_result_df.iloc[:,:-1]

#target
y=final_result_df.iloc[:,-1]

# Specify the hyperparameter space, if we increase the penalty(c) and gamma value the accurancy can be increased.
#since it takes lots of time in google colab provided only a single value
parameters = {'SVM__C':[1, 10, 100],
              'SVM__gamma':[0.1, 0.01]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=21)

cv = GridSearchCV(pipeline,parameters,cv=3)
cv.fit(X_train,y_train)

y_pred = cv.predict(X_test)
SVM_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
SVM_Accuracy=cv.score(X_test, y_test)

print("Accuracy: {}".format(SVM_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.8556034482758621
              precision    recall  f1-score   support

           0       0.65      0.97      0.78       119
           1       1.00      1.00      1.00       125
           2       0.96      0.79      0.86       140
           3       1.00      0.57      0.73        80

    accuracy                           0.86       464
   macro avg       0.90      0.83      0.84       464
weighted avg       0.90      0.86      0.86       464

Tuned Model Parameters: {'SVM__C': 10, 'SVM__gamma': 0.01}


### **XGBOOST**

In [43]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [44]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [45]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {}".format(accuracy))
print(classification_report(y_test, y_pred))

Accuracy: 0.8620689655172413
              precision    recall  f1-score   support

           0       0.79      0.76      0.78       119
           1       0.98      1.00      0.99       125
           2       0.84      0.87      0.85       140
           3       0.82      0.78      0.79        80

    accuracy                           0.86       464
   macro avg       0.86      0.85      0.85       464
weighted avg       0.86      0.86      0.86       464



#### **SAVING A VERY BASIC ML MODEL AND USING IT ON REALTIME PIPELINE TO CHECK WORKING**


In [46]:
# Import the necessary modules for ML model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
import joblib
#input
X = final_result_df.iloc[:,:-1]
#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

knn =  KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train,y_train)

joblib_file='knn_model_test.pkl'
joblib.dump(knn,joblib_file)

['knn_model_test.pkl']

In [47]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

#input
X = pd.read_csv('final_1D.csv',header=None)

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=21)

svm=SVC(C=10,gamma=0.01)

svm.fit(X_train,y_train)

joblib_file='svm_model_test.pkl'
joblib.dump(svm,joblib_file)

['svm_model_test.pkl']

### **ENSEMBLE**


In [ ]:
# Importing required modules
from sklearn import linear_model, tree, ensemble
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import  XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
#input
X = final_result_df.iloc[:,0:-1]

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test,  y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [50]:
# Stacking of ML Models
eclf = VotingClassifier(estimators=[
    ('SVM', SVC(probability=True)),
    ('knn', KNeighborsClassifier()),
    ('rf', ensemble.RandomForestClassifier()),
    ('bayes',GaussianNB()),
    ('logistic',LogisticRegression()),
    ], voting='soft')

In [51]:
# Hyperparameter Tuning using gridSearch
params = {'SVM__C':[1, 10, 100],
          'SVM__gamma':[0.1, 0.01],
          'knn__n_neighbors': [1,3,5],
          'rf__n_estimators':[300, 400],
          }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
voting_clf = grid.fit(X_train, y_train)

print(grid.best_params_)
y_pred = voting_clf.predict(X_test)

# Compute and print metrics
Voting_Accuracy=voting_clf.score(X_test, y_test)

print("Accuracy: {}".format(Voting_Accuracy))
print(classification_report(y_test, y_pred))
print(voting_clf.best_params_)

{'SVM__C': 100, 'SVM__gamma': 0.1, 'knn__n_neighbors': 5, 'rf__n_estimators': 300}
Accuracy: 0.9032258064516129
              precision    recall  f1-score   support

           0       0.97      0.94      0.96        80
           1       1.00      1.00      1.00        72
           2       0.81      0.87      0.84        79
           3       0.80      0.75      0.77        48

    accuracy                           0.90       279
   macro avg       0.90      0.89      0.89       279
weighted avg       0.90      0.90      0.90       279

{'SVM__C': 100, 'SVM__gamma': 0.1, 'knn__n_neighbors': 5, 'rf__n_estimators': 300}


In [53]:
# open a file, where you ant to store the data
file = open('Heart_Disease_Prediction_using_image_processing__ECG.pkl', 'wb')
# dump information to that file
pickle.dump(voting_clf, file)

In [2]:
import sklearn

print(sklearn.__version__)


1.3.2
